# TP6

### `Resolver usando Pandas`

Resolver los ejercicios del TP3 utilizando la librería Pandas.

### Ejercicio 1: Cargar Datos de ventas.

El archivo datos.dat contiene el registro de las ventas realizadas.

Tiene un formato de ancho fijo:
- `fecha`:    10 lugares
- `producto`: 30 lugares
- `precio`:   10 lugares
- `cantidad`:  5 lugares

Hacer una funcion que cargue los datos en un DataFrame de Pandas.

In [1]:
import pandas as pd

def cargar_datos():
    anchos = [10, 30, 10, 5]

    nombres = ['fecha', 'producto', 'precio', 'cantidad']

    df = pd.read_fwf('datos.dat', widths=anchos, names=nombres)

    df['fecha'] = pd.to_datetime(df['fecha'])

    df['precio'] = df['precio'].astype(float)
    df['cantidad'] = df['cantidad'].astype(int)

    return df

datos = cargar_datos()

print(datos)

        fecha    producto  precio  cantidad
0  2024-08-27     Mirinda  1510.0        14
1  2024-08-27     Mirinda  1560.0        12
2  2024-08-28     Torasso   940.0         8
3  2024-08-29  Pepsi Cola  1210.0        10
4  2024-08-30     Mirinda  1520.0         1
5  2024-09-01     Mirinda  1550.0        15
6  2024-09-01      Sprite   810.0         4
7  2024-09-02   Coca Cola  1100.0         4
8  2024-09-02  Pepsi Cola  1220.0        13
9  2024-09-02     Torasso   910.0         5
10 2024-09-02     Torasso   920.0         3
11 2024-09-03   Coca Cola  1020.0         8
12 2024-09-03     Mirinda  1570.0         7
13 2024-09-03     Mirinda  1590.0         2
14 2024-09-04  Pepsi Cola  1220.0        13
15 2024-09-05     Mirinda  1500.0         3
16 2024-09-05  Pepsi Cola  1300.0         5
17 2024-09-06   Coca Cola  1080.0         1
18 2024-09-06      Sprite   860.0        12
19 2024-09-06     Torasso   930.0         3
20 2024-09-07   Coca Cola  1080.0        14
21 2024-09-07      Sprite   870.

### Ejercicio 2: Calcular el total de ventas.
Hacer una función que sume los importes vendidos (precio * cantidad) y las cantidades.


In [2]:
def calcular_totales(datos):
    datos['importe'] = datos['precio'] * datos['cantidad']
    
    importe_total = datos['importe'].sum()
    
    cantidad_total = datos['cantidad'].sum()
    
    return importe_total, cantidad_total

importe, cantidad = calcular_totales(datos)

print(f"Las ventas fueron de ${importe:.2f} en {cantidad} unidades")

Las ventas fueron de $392730.00 en 335 unidades


### Ejercicio 3: Listar las unidades vendidas.
Listar cuántas unidades se vendieron en total para cada producto


In [3]:
def unidades_vendidas(datos):
    ventas_por_producto = datos.groupby('producto') ['cantidad'].sum()

    ventas_ordenadas = ventas_por_producto.sort_values (ascending = False)

    print("Unidades vendidas por producto:")
    for producto, cantidad in ventas_ordenadas.items():
        print(f"{producto}: {cantidad} unidades")

unidades_vendidas(datos)

Unidades vendidas por producto:
Pepsi Cola: 89 unidades
Mirinda: 85 unidades
Sprite: 72 unidades
Coca Cola: 57 unidades
Torasso: 32 unidades


###  Ejercicio 4: Listar el precio promedio por producto.
Hacer un listado del precio promedio por producto.


In [4]:
def precio_promedio(datos):
    promedio_por_producto = datos.groupby('producto')['precio'].mean()
    
    promedio_ordenado = promedio_por_producto.sort_values(ascending=False)
    
    print("Precio promedio por producto:")
    for producto, precio in promedio_ordenado.items():
        print(f"{producto}: ${precio:.2f}")
    
precio_promedio(datos)

Precio promedio por producto:
Mirinda: $1545.83
Pepsi Cola: $1245.00
Coca Cola: $1072.50
Torasso: $920.00
Sprite: $841.43


### Ejercicio 5: Ranking de productos
Realizar un listado de los 3 productos más vendidos ordenados por la cantidad de unidades vendidas (ordenadas de mayor a menor)


In [5]:
def ranking_productos(datos, top=3):
    ventas_por_producto = datos.groupby('producto')['cantidad'].sum()
    
    top_productos = ventas_por_producto.sort_values(ascending=False).head(top)
    
    print(f"Los {top} productos más vendidos son:")
    for i, (producto, cantidad) in enumerate(top_productos.items(), 1):
        print(f"{i}. {producto}: {cantidad} unidades")
    

ranking_productos(datos)

Los 3 productos más vendidos son:
1. Pepsi Cola: 89 unidades
2. Mirinda: 85 unidades
3. Sprite: 72 unidades


### Ejercicio 6: Listar las ventas por mes
Realizar un listado del total de unidades vendidas por producto separado por mes.


In [6]:
def ventas_por_mes(datos):
    datos['fecha'] = pd.to_datetime(datos['fecha'])

    datos['año'] = datos['fecha'].dt.year
    datos['mes'] = datos['fecha'].dt.month

    ventas_agrupadas = datos.groupby(['año', 'mes', 'producto'])['cantidad'].sum().unstack(fill_value=0)

    ventas_agrupadas = ventas_agrupadas.sort_index()

    for (año, mes), fila in ventas_agrupadas.groupby(level=[0,1]):
        print(f"\nVentas en {año}-{mes:02d}:")
        for producto in fila.columns:
            cantidad = fila[producto].values[0]
            if cantidad > 0:
                print(f" {producto}: {cantidad} unidades")

ventas_por_mes(datos)


Ventas en 2024-08:
 Mirinda: 27 unidades
 Pepsi Cola: 10 unidades
 Torasso: 8 unidades

Ventas en 2024-09:
 Coca Cola: 57 unidades
 Mirinda: 58 unidades
 Pepsi Cola: 79 unidades
 Sprite: 72 unidades
 Torasso: 24 unidades


### Ejercicio 7: Informe general

Mostrar un listado de productos ordenados alfabeticamente que contengan el precio promedio, la cantidad de unidades vendidas y el importe total vendido para cada producto

In [7]:
def resumen_ventas(datos):
    datos['importe_total'] = datos['precio'] * datos['cantidad']
    
    resumen = datos.groupby('producto').agg({
        'precio': 'mean',
        'cantidad': 'sum',
        'importe_total': 'sum'
    })
    
    resumen.columns = ['Precio Promedio', 'Unidades Vendidas', 'Importe Total']
    
    resumen = resumen.sort_index()
    
    print("Resumen de ventas por producto:")
    for producto, fila in resumen.iterrows():
        print(f"\n{producto}:")
        print(f"  Precio Promedio: ${fila['Precio Promedio']:.2f}")
        print(f"  Unidades Vendidas: {fila['Unidades Vendidas']}")
        print(f"  Importe Total: ${fila['Importe Total']:.2f}")

resumen_ventas(datos)

Resumen de ventas por producto:

Coca Cola:
  Precio Promedio: $1072.50
  Unidades Vendidas: 57.0
  Importe Total: $60780.00

Mirinda:
  Precio Promedio: $1545.83
  Unidades Vendidas: 85.0
  Importe Total: $131080.00

Pepsi Cola:
  Precio Promedio: $1245.00
  Unidades Vendidas: 89.0
  Importe Total: $110510.00

Sprite:
  Precio Promedio: $841.43
  Unidades Vendidas: 72.0
  Importe Total: $61040.00

Torasso:
  Precio Promedio: $920.00
  Unidades Vendidas: 32.0
  Importe Total: $29320.00


## `Resolver usando NumPy`
## Resolver el ejercicio 2 del tp1 usando NumPy

### Ejercicio 8

Escribe una función en Python que encuentre los valores de `a`, `b`, y `c` para que la función cuadrática `f(x) = a x^2 + b x + c` pase exactamente por los siguientes puntos:

| x  | y  |
|---:|---:|
|  0 |  0 |
|  1 |  8 |
|  2 | 12 |
|  3 | 12 |
|  5 |  0 |

### Requisitos:
- La función debe explorar posibles valores de `a`, `b`, y `c` utilizando un método de prueba y error.
- Debe devolver los valores que hagan que la diferencia entre la función `f(x)` y los valores medidos `y` sea exactamente cero para cada punto.

> **Pista**: Los valores de `a`, `b`, y `c` son números pequeños.

La idea es implementar el mismo algoritmo que se uso en el TP1 pero usando NumPy en lugar de Python puro.

In [8]:
import numpy as np

def f(x, coeficientes):
    a, b, c = coeficientes
    return a*x**2 + b*x + c

def error(y, y_pred):
    return np.sum(np.abs(y - y_pred))

X = np.array([0, 1, 2, 3, 5])
Y = np.array([0, 8, 12, 12, 0])

def buscar_coeficientes():
    mejor_error = float('inf')
    mejores_coeficientes = None
    
    for a in np.arange(-5, 5, 0.1):
        for b in np.arange(-20, 20, 0.1):
            for c in np.arange(-10, 10, 0.1):
                coeficientes = (a, b, c)
                y_pred = f(X, coeficientes)
                error_actual = error(Y, y_pred)
                
                if error_actual < mejor_error:
                    mejor_error = error_actual
                    mejores_coeficientes = coeficientes
                
                if np.isclose(error_actual, 0, atol=1e-6):
                    return coeficientes
    
    return mejores_coeficientes

coeficientes = buscar_coeficientes()
print(f"Coeficientes encontrados: a = {coeficientes[0]:.2f}, b = {coeficientes[1]:.2f}, c = {coeficientes[2]:.2f}")

y_pred = f(X, coeficientes)
print("\nVerificación:")
for x, y, y_p in zip(X, Y, y_pred):
    print(f"x = {x}, y real = {y}, y predicho = {y_p:.2f}")

print(f"\nError total: {error(Y, y_pred):.6f}")

Coeficientes encontrados: a = -2.00, b = 10.00, c = -0.00

Verificación:
x = 0, y real = 0, y predicho = -0.00
x = 1, y real = 8, y predicho = 8.00
x = 2, y real = 12, y predicho = 12.00
x = 3, y real = 12, y predicho = 12.00
x = 5, y real = 0, y predicho = 0.00

Error total: 0.000000


### Ejercicio 9: Resolver el ejercicio 3 del TP1 usando NumPy
Buscar los coeficientes de la función que minimice la suma de los cuadrados de las diferencias entre los valores medidos y los valores de la función.

1. Crear un array con los coeficientes elegidos al azar (usar `randint(-10,10,3)`).
2. Calcular el valor de la función y el error correspondiente.
3. Mientras que el error sea mayor a 1:
    1. Definir nuevos coeficientes agregándoles un pequeño valor al azar a los coeficientes actuales (aprendizaje = 0.001).
    2. Si el error para los nuevos coeficientes es menor que el anterior, reemplazar los coeficientes actuales por los nuevos.


In [9]:
import numpy as np
from numpy.random import randint

def f(x, coeficientes):
    a, b, c = coeficientes
    return a*x**2 + b*x + c

def error(y, y_pred):
    return np.sum((y - y_pred)**2)

X = np.array([0, 1, 2, 3, 5])
Y = np.array([0, 8, 12, 11, 1])

def buscar_coeficientes():
    coeficientes = randint(-10, 10, 3)
    
    y_pred = f(X, coeficientes)
    error_actual = error(Y, y_pred)
    
    while error_actual > 1:
        aprendizaje = 0.001
        nuevos_coeficientes = coeficientes + randint(-10, 10, 3) * aprendizaje
        
        nuevos_y_pred = f(X, nuevos_coeficientes)
        nuevo_error = error(Y, nuevos_y_pred)
        
        if nuevo_error < error_actual:
            coeficientes = nuevos_coeficientes
            error_actual = nuevo_error
    
    return coeficientes

coeficientes = buscar_coeficientes()

print("Coeficientes encontrados:")
print(f"a = {coeficientes[0]:.4f}")
print(f"b = {coeficientes[1]:.4f}")
print(f"c = {coeficientes[2]:.4f}")

y_pred = f(X, coeficientes)
error_final = error(Y, y_pred)
print(f"\nError final: {error_final:.4f}")

print("\nComparación de valores:")
print("x  |  y real   |  y predicho")
for x, y_real, y_p in zip(X, Y, y_pred):
    print(f"{x}  |  {y_real:7.2f}  |  {y_p:10.2f}")

Coeficientes encontrados:
a = -1.7820
b = 8.9610
c = 0.6140

Error final: 0.9982

Comparación de valores:
x  |  y real   |  y predicho
0  |     0.00  |        0.61
1  |     8.00  |        7.79
2  |    12.00  |       11.41
3  |    11.00  |       11.46
5  |     1.00  |        0.87
